In [ ]:
# Install required packages
!pip install -q spotdl ipywidgets

import os
import shutil
import datetime
import ipywidgets as widgets
from IPython.display import display
from google.colab import files, drive
from subprocess import run, CalledProcessError

# Mount Google Drive
drive.mount('/content/drive')

# UI Elements
url_input = widgets.Text(
    value='Please input your Spotify link here...',
    placeholder='e.g. https://open.spotify.com/track/3n3Ppam7vgaVa1iaRUc9Lp',
    description='Spotify URL:',
    layout=widgets.Layout(width='100%')
)

upload_checkbox = widgets.Checkbox(
    value=False,
    description='Upload to Google Drive'
)

download_checkbox = widgets.Checkbox(
    value=True,
    description='Download ZIP to local device'
)

download_button = widgets.Button(
    description='Download & ZIP',
    button_style='success'
)

output = widgets.Output()

def download_music(b):
    with output:
        output.clear_output()
        print("🎵 Starting download...")

        # Prepare directories
        music_dir = "/content/music"
        shutil.rmtree(music_dir, ignore_errors=True)
        os.makedirs(music_dir, exist_ok=True)
        os.chdir(music_dir)

        try:
            result = run(["spotdl", url_input.value], check=True)
            print("✅ Download complete.")

            # Create timestamped ZIP filename
            timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
            zip_name = f"music_{timestamp}.zip"
            zip_path = f"/content/{zip_name}"

            # Zip the music folder
            os.chdir("/content")
            shutil.make_archive("music", "zip", music_dir)
            os.rename("/content/music.zip", zip_path)
            print(f"📦 Music zipped successfully → {zip_name}")

            # Upload to Google Drive
            if upload_checkbox.value:
                drive_path = "/content/drive/MyDrive/spotdl_downloads"
                os.makedirs(drive_path, exist_ok=True)
                shutil.copy(zip_path, drive_path)
                print(f"☁️ Uploaded to Google Drive → {drive_path}/{zip_name}")

            # Download to local device
            if download_checkbox.value:
                print("⬇️ Preparing file download...")
                files.download(zip_path)

            print("✅ All done! Enjoy your music 🎶")

        except CalledProcessError:
            print("❌ Error during download. Please check the Spotify URL and try again.")

# Bind button
download_button.on_click(download_music)

# Display interface
display(url_input, upload_checkbox, download_checkbox, download_button, output)
